In [1]:
from warnings import simplefilter 
simplefilter(action='ignore', category=FutureWarning)
import random
from  keras.preprocessing.image import save_img
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
import io
import os
import yaml
from Util import *
from AdversarialAttack import *
from IPython import display
import copy
%matplotlib inline
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11709157902769039359
]


In [13]:
import keras
model = keras.models.load_model('./Models/Mnist')
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_test = x_test.astype("float32") / 255
x_test = np.expand_dims(x_test, -1)
y_test = keras.utils.to_categorical(y_test, 10)
test = np.expand_dims(x_test[0],0)
print(test.shape)
print(model.predict(test))

(1, 28, 28, 1)
[[5.7923527e-10 5.8396510e-11 2.0652860e-06 3.9138008e-06 3.2948291e-14
  1.2028812e-09 5.7205052e-17 9.9999380e-01 2.5606195e-09 2.7493741e-07]]


In [3]:
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
print(x_train.shape)

(60000, 28, 28, 1)


In [5]:

from tensorflow.keras import layers
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [6]:
batch_size = 128
epochs = 15
with tf.device('/GPU:0'):
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/15
422/422 [==============================] - 11s 26ms/step - loss: 0.7687 - accuracy: 0.7634 - val_loss: 0.0791 - val_accuracy: 0.9785
Epoch 2/15
422/422 [==============================] - 11s 26ms/step - loss: 0.1233 - accuracy: 0.9635 - val_loss: 0.0547 - val_accuracy: 0.9850
Epoch 3/15
422/422 [==============================] - 12s 27ms/step - loss: 0.0855 - accuracy: 0.9740 - val_loss: 0.0507 - val_accuracy: 0.9862
Epoch 4/15
422/422 [==============================] - 12s 27ms/step - loss: 0.0688 - accuracy: 0.9792 - val_loss: 0.0434 - val_accuracy: 0.9897
Epoch 5/15
422/422 [==============================] - 10s 25ms/step - loss: 0.0604 - accuracy: 0.9815 - val_loss: 0.0439 - val_accuracy: 0.9892
Epoch 6/15
422/422 [==============================] - 10s 24ms/step - loss: 0.0570 - accuracy: 0.9817 - val_loss: 0.0406 - val_accuracy: 0.9890
Epoch 7/15
422/422 [==============================] - 10s 23ms/step - loss: 0.0535 - accuracy: 0.9833 - val_loss: 0.0347 - val_accuracy:

In [7]:
model.save('./Models/Mnist')


INFO:tensorflow:Assets written to: ./Models/Mnist/assets
